In [121]:
### Create interactive map of residents living within 400 - 1600 m (+ 250 m because of the grid resolution) walking distance from 
### metro and train station in Finland's capital region with a value slider. Data for the map is loaded from open data services of 
### Maanmittauslaitos (MML) and Helsinki Region Environmental Services Authority (HSY).

### IMPORT DATA ###

# Import modules
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from geopandas.tools import geocode
import numpy as np
from pyproj import CRS
import requests
import geojson
import matplotlib.pyplot as plt
from shapely.ops import cascaded_union
import mapclassify
import contextily as ctx
from mpl_toolkits.axes_grid1 import make_axes_locatable

## Read shape file containing the capital region as polygons into variable 'grid' (Data from https://tiedostopalvelu.maanmittauslaitos.fi/tp/kartta)
# File path
fp_grid = "data/pkseutu.shp"

# Read in data
grid = gpd.read_file(fp_grid)

# Check if crs is correct and set crs to ETRS89 / TM35FIN if the crs is not defined correctly
if (grid.crs != "epsg:3067"):    
    grid = grid.set_crs(epsg=3067)
# Reproject to WGS 84 / Pseudo-Mercator if the crs is not defined correctly
if (grid.crs != "epsg:3857"):    
    grid = grid.to_crs(epsg=3857)

# Combine polygons of each city to form one polygon of the whole capial region
grid['constant'] = 0
boundary = grid.dissolve(by='constant')

# Check the data
print(grid.head())
print(grid.crs)
print(boundary)

     GML_ID NATCODE     NAMEFIN      NAMESWE  LANDAREA  FRESHWAREA  SEAWAREA  \
0  27817426     091    Helsinki  Helsingfors    214.25        0.91    500.32   
1  27806267     049       Espoo         Esbo    312.32       17.91    197.80   
2  27817421     092      Vantaa        Vanda    238.37        1.98      0.00   
3  27806296     235  Kauniainen    Grankulla      5.89        0.11      0.00   

   TOTALAREA                                           geometry  constant  
0     715.48  POLYGON ((2804165.799 8463618.196, 2804168.439...         0  
1     528.03  POLYGON ((2764219.067 8456519.067, 2764250.094...         0  
2     240.35  POLYGON ((2804165.799 8463618.196, 2804091.743...         0  
3       6.00  POLYGON ((2753937.589 8448624.411, 2753958.644...         0  
epsg:3857
                                                   geometry    GML_ID NATCODE  \
constant                                                                        
0         POLYGON ((2804165.799 8463618.196, 28

In [122]:
## Read population grid data for 2018 into a variable `pop`. 

# Specify the url for web feature service
url = 'https://kartta.hsy.fi/geoserver/wfs'

# Specify parameters (read data in json format).
params = dict(service='WFS',
              version='2.0.0',
              request='GetFeature',
              typeName='asuminen_ja_maankaytto:Vaestotietoruudukko_2018',
              outputFormat='json')

# Fetch data from WFS using requests
r = requests.get(url, params=params)

# Create GeoDataFrame from geojson
pop = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

# Clean out unnecessary columns
pop = pop[["asukkaita", "geometry"]]

# Set crs to ETRS89 / GK25FIN and reproject to WGS 84 / Pseudo-Mercator if the crs is not defined correctly
if (pop.crs == None):    
    pop = pop.set_crs(epsg=3879)
if (pop.crs != "epsg:3857"):    
    pop = pop.to_crs(epsg=3857)

# Check the data
print(pop.head())
print(pop.crs)

   asukkaita                                           geometry
0        108  MULTIPOLYGON Z (((2735848.290 8440015.529 0.00...
1        273  MULTIPOLYGON Z (((2736346.567 8440521.047 0.00...
2        239  MULTIPOLYGON Z (((2736832.315 8443036.322 0.00...
3        202  MULTIPOLYGON Z (((2736835.465 8442533.836 0.00...
4        261  MULTIPOLYGON Z (((2736838.614 8442031.385 0.00...
epsg:3857


In [123]:
## Read buffer polygons that describe 400 m, 800 m, 1200 m and 1600 m accessibilities via pedestrian and bicycle ways from metro and 
## train stations 

# Save wanted buffer sizes in a list which is used in loading the data
dists = ['400', '800', '1200', '1600']

# Create an empty geopandas GeoDataFrame for the data
buffs = gpd.GeoDataFrame()

# Iterate through wanted buffer distance list
for dist in dists:

    # Specify the url for web feature service and typeName of the data layer
    url_buff = 'https://kartta.hsy.fi/geoserver/wfs'
    type_name = dist + 'm_verkostobufferi'

    # Specify parameters (read data in json format).
    params_buff = dict(service='WFS',
                  version='2.0.0',
                  request='GetFeature',
                  typeName=type_name,
                  outputFormat='json')

    # Fetch data from WFS using requests
    r = requests.get(url_buff, params=params_buff)

    # Create GeoDataFrame from geojson
    buff = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

    # Clean out unnecessary columns
    buff = buff[["asema", "geometry"]]

    # Set crs to ETRS89 / GK25FIN and reproject to WGS 84 / Pseudo-Mercator if the crs is not defined correctly
    if (buff.crs == None):    
        buff = buff.set_crs(epsg=3879)
    if (buff.crs != "epsg:3857"):    
        buff = buff.to_crs(epsg=3857)

    # Clip out stations that are located outside the capital region
    clip_mask = buff.within(boundary.at[0,'geometry'])
    buff = buff.loc[clip_mask]
    
    # Create column which indicates buffer distance for the slider
    buff['dist'] = dist

    # Check the data
    print(buff.head(1))
    print(len(buff))

    # Add the data to combined GeoDataFrame
    buffs = buffs.append(buff)
    
# Check output of the loop
buffs.head()

    asema                                           geometry dist
0  Käpylä  MULTIPOLYGON (((2777311.378 8449582.555, 27773...  400
64
  asema                                           geometry dist
0  Kilo  MULTIPOLYGON (((2759279.008 8447422.698, 27592...  800
64
       asema                                           geometry  dist
1  Myllypuro  MULTIPOLYGON (((2791802.822 8452130.551, 27917...  1200
64
  asema                                           geometry  dist
0  Kera  MULTIPOLYGON (((2753229.459 8448403.562, 27532...  1600
63


,asema,geometry,dist
0,Käpylä,"MULTIPOLYGON (((2777311.378 8449582.555, 27773...",400
1,Lentoasema,"MULTIPOLYGON (((2779250.240 8469657.438, 27792...",400
2,Hiekkaharju,"MULTIPOLYGON (((2788740.833 8468060.732, 27887...",400
4,Helsinki,"MULTIPOLYGON (((2776468.702 8439075.592, 27764...",400
6,Kannelmäki,"MULTIPOLYGON (((2768908.228 8453913.003, 27689...",400


In [124]:
### PROCESS DATA ###

# Create new column to 'buffs' where total resident amounts within each buffer areas are stored 
buffs["residents_sum"] = None

# Create a spatial join between grid layer and buffer layer. "Intersects" option used here to include all grid cells which 
# touch the buffer area (NOTE that with this choice the accuracy of the buffers is lost due to the grid resolution)
pop_combined = gpd.sjoin(pop, buffs, how="left", op="intersects")

# Group the data by both train and metro station names AND distance classes
groupedA = pop_combined.groupby(['asema','dist'])

#buffs.head()
#pop_combined.head()
groupedA.head()

,asukkaita,geometry,index_right,asema,dist,residents_sum
0,108,MULTIPOLYGON Z (((2735848.290 8440015.529 0.00...,49.0,Mankki,400,None
0,108,MULTIPOLYGON Z (((2735848.290 8440015.529 0.00...,4.0,Mankki,800,None
0,108,MULTIPOLYGON Z (((2735848.290 8440015.529 0.00...,72.0,Mankki,1200,None
1,273,MULTIPOLYGON Z (((2736346.567 8440521.047 0.00...,49.0,Mankki,400,None
1,273,MULTIPOLYGON Z (((2736346.567 8440521.047 0.00...,4.0,Mankki,800,None
...,...,...,...,...,...,...
3119,866,MULTIPOLYGON Z (((2798045.186 8446684.536 0.00...,21.0,Vuosaari,400,None
3120,720,MULTIPOLYGON Z (((2798044.157 8446181.791 0.00...,21.0,Vuosaari,400,None
3121,1075,MULTIPOLYGON Z (((2798043.129 8445679.080 0.00...,21.0,Vuosaari,400,None
3122,1164,MULTIPOLYGON Z (((2798042.101 8445176.403 0.00...,21.0,Vuosaari,400,None


In [127]:
# Store sum of residents living approximately 400 m, 800 m, 1200 m and 1600 m from station to column "sum" 
# (the distance doesn't stay constant in performed analysis but accurate enough for this visualization)
for name, group in groupedA:
    #for dist in dists:
        #buffs['residents_sum'].at[buffs["asema"]==name and buffs['dist']==dist] = group["asukkaita"].agg("sum")
    #print(name)
    #group_name = name[0]
    #group_dist =
    buffs.loc[(buffs["asema"]==name[0]) & (buffs['dist']==name[1]),'residents_sum'] = group["asukkaita"].agg("sum")
    #print(group)
    
## Convert the buffer polygons to points (location set as centroids of 400 m buffers, approximate of the station locations)
point_data = buffs

##for name in points['dist']=='400': 
##    for dist in dists:
##        points['geometry'].loc[(points['asema']==name) & (points['dist']==dist)] = points.at[points['asema']==name and points['dist']=='400'].centroid

# Group the data by only train and metro station names
groupedB = point_data.groupby('asema').first()

# Convert to points based on 400 m buffer centroids
for i in groupedB:
    
    point_data.loc[point_data["asema"]==i,'geometry'] = i[1].centroid
    
# Reorganize the column order
#point_data = point_data[["geometry","asema","residents_sum", "dist"]]

# Replace NoData in residents_sum column with 0
#point_data["residents_sum"] = point_data["residents_sum"].replace(to_replace=np.nan, value=0)
    
# Check data    
#print(point_data.head())
#type(point_data)
groupedB.tail()


geometry
dist
residents_sum


,geometry,dist,residents_sum
asema,,,
Urheilupuisto,"MULTIPOLYGON (((2758227.841 8439114.083, 27582...",400,2070.0
Valimo,"MULTIPOLYGON (((2769068.017 8449931.672, 27690...",400,807.0
Vantaankoski,"MULTIPOLYGON (((2766279.148 8464198.121, 27662...",400,310.0
Vehkala,"MULTIPOLYGON (((2765961.788 8465860.373, 27659...",400,NaN
Vuosaari,"MULTIPOLYGON (((2798968.088 8446567.971, 27989...",400,7402.0


In [110]:
print(point_data.loc[(point_data['dist']=='400') & (point_data['asema']=='Malmi')])
print(point_data.loc[(point_data['dist']=='800') & (point_data['asema']=='Malmi')])

                           geometry  asema  residents_sum dist
67  POINT (2784231.092 8455859.270)  Malmi           3437  400
                           geometry  asema  residents_sum dist
79  POINT (2784237.773 8455858.476)  Malmi           9573  800
